In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
from PIL import Image
import torchvision.models as models

from src.yolov8 import *
from src.yoloTrain import *
from src.yoloDataset import *
from src.yoloLoss import *
from src.utils import *

In [ ]:
from src.datasetSetup import srediDataset
srediDataset("datasetoviraw", "datasetyolo.zip")

In [ ]:
model = YOLOv8(features)
optimizer = optim.Adam(model.fc_layers.parameters(), lr=0.001)

train_dataset = YoloV8Dataset(train_img_dir, train_labels_dir, transform=transform_yolo)
train_loader = DataLoader(train_dataset, batch_size=batch_size_yolo, shuffle=True, num_workers=2, collate_fn=collate_fn)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [ ]:

for epoch in range(num_epochs_yolo):
    epoch_loss = train(model, optimizer, train_loader, device)
    print(f'Epoch {epoch+1}/{num_epochs_yolo}, Loss: {epoch_loss:.4f}')